# Coding Exercise

You must correctly implement the function described in the prompt below.

Feel free to test out pieces of code to help you write the solution.

Please thoroughly test that the final code implements the function correctly.

## Prompt

**Function signature:** `seat(R: int, C: int, A: int, B: int) -> List[str]`

    
    You are running a cinema.
    The auditorium is a rectangular array of seats with R rows and C columns in each row.

    Due to the Covid-19 regulations, people who are not a part of the same group are not allowed to sit next to each other, nor immediately behind each other.

    Two groups of people would like to see the same movie.
    There are A people in the first group and B people in the second group.

    If it is impossible to seat all those people according to the regulations, return an empty String[].
    Otherwise, return any one possible seating plan: a String[] with R elements, each containing C characters. Use 'A' for people from the first group, 'B' for people from the second group, and '.' for empty seats.

    Constraints
    -R will be between 1 and 50, inclusive.
    -C will be between 1 and 50, inclusive.
    -A will be between 1 and R*C, inclusive.
    -B will be between 1 and R*C, inclusive.
 
    Examples
    0)
        4
    6
    2
    3

    Returns: {"......", ".A..BB", "......", "B.A..." }
    A rather large cinema and very few people. We have many ways to place them. The example one is shown below in a nicer format:

    ......
    .A..BB
    ......
    B.A...

    1)
        4
    5
    13
    7

    Returns: { }
    These two groups would fill the cinema completely, and that's not possible without some 'A' being adjacent to some 'B'.

    2)
        4
    5
    14
    2

    Returns: {"B.AAA", ".AAAA", "AAAA.", "AAA.B" }

    B.AAA
    .AAAA
    AAAA.
    AAA.B

    3)
        5
    5
    17
    4

    Returns: {"AAAAA", "AAAAA", "..AAA", "BB.AA", "BB.AA" }

    AAAAA
    AAAAA
    ..AAA
    BB.AA
    BB.AA

    

One possible solution is to put everyone of the same group together and check the if any A is adjacent to a B.

If group everyone together, we can find a solution that minimizes the surface of the group with the least number of people (say group 'A') and fit the rest (group 'B') anywhere except by the intersections.

How can we minimize the surface? I will minimize the surface by adding one person at a time, starting from a corner.

First, I will write a function to calculate the surface (number of neighbors 'A' and 'B' or 0 and 1)

In [1]:
from numpy import zeros

def calculate_surface(state):
    surface = 0
    
    x, y = state.shape
    for i in range(x):
        for j in range(y):
            if state[i,j] == 0:
                continue
                
            if i+1 < x  and state[i+1,j  ] == 0:
                surface += 1
            if j+1 < y  and state[i  ,j+1] == 0:
                surface += 1
            if i-1 >= 0 and state[i-1,j  ] == 0:
                surface += 1
            if j-1 >= 0 and state[i  ,j-1] == 0:
                surface += 1
    
    return surface

#only one person at the corner
s = zeros([5,4])
s[0,0] = 1

print(s)
print(calculate_surface(s))

[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
2


It is working in this case. Let me add  a few more people

In [3]:
s[4,3] = 1
s[1,1] = 1
print(s)
print(calculate_surface(s))

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]
8


The function is counting duplicates. Let me remove the duplicate couting by summing over empty spaces (0).

In [4]:
def calculate_surface(state):
    surface = 0
    
    x, y = state.shape
    for i in range(x):
        for j in range(y):
            if state[i,j] == 1:
                continue
                
            if i+1 < x  and state[i+1,j  ] == 1:
                surface += 1
                continue
            if j+1 < y  and state[i  ,j+1] == 1:
                surface += 1
                continue
            if i-1 >= 0 and state[i-1,j  ] == 1:
                surface += 1
                continue
            if j-1 >= 0 and state[i  ,j-1] == 1:
                surface += 1
                continue
    
    return surface

print(s)
print(calculate_surface(s))

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]
6


Now it is working.

Before I implement the seat function, it may be interesting to get the indexes at the surface because we increase the surface by the least ammount if we add a person by side-by-side.

In [7]:
def surface(state):
    surface = []
    
    x, y = state.shape
    for i in range(x):
        for j in range(y):
            if state[i,j] == 1:
                continue
                
            if i+1 < x  and state[i+1,j  ] == 1:
                surface.append([i, j])
                continue
            if j+1 < y  and state[i  ,j+1] == 1:
                surface.append([i, j])
                continue
            if i-1 >= 0 and state[i-1,j] == 1:
                surface.append([i, j])
                continue
            if j-1 >= 0 and state[i,j-1] == 1:
                surface.append([i, j])
                continue
    
    return surface

print(surface(s))

[[0, 1], [1, 0], [1, 2], [2, 1], [3, 3], [4, 2]]


it is working.

Now we implement a function that, given R , C and A, it will minimize the surface and return the state of the theater containing the peoples position.

In [16]:
def minimize_surface(R, C, A, state=None):
    
    if state is None:
        state = zeros([R, C])
        
    state[0,0] = 1
    
    for i in range(1, A):
        surface_idx = surface(state)
        surface_list = []
        
        for s in surface_idx:
            
            state[s[0],s[1]] = 1
            surface_list.append(calculate_surface(state))
            state[s[0],s[1]] = 0
            
        s_min = surface_list.index(min(surface_list))
        s_min = surface_idx[s_min]
        
        state[s_min[0], s_min[1]] = 1
        
    return state
            
minimize_surface(5, 5, 4)

array([[1., 1., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [17]:
minimize_surface(5, 5, 5)

array([[1., 1., 1., 0., 0.],
       [1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [18]:
minimize_surface(5, 5, 6)

array([[1., 1., 1., 0., 0.],
       [1., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [19]:
minimize_surface(5, 5, 10)

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [20]:
minimize_surface(5, 5, 11)

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

All the results above seems to return the correct result.

Now, let me define a function to fill the other people.

In [29]:
def fill_state(state, B):
    
    x, y = state.shape
    
    for i in range(x):
        for j in range(y):
            if state[i,j] == 1:
                continue
                
            if i+1 < x  and state[i+1,j  ] == 1:
                continue
            if j+1 < y  and state[i  ,j+1] == 1:
                continue
            if i-1 >= 0 and state[i-1,j] == 1:
                continue
            if j-1 >= 0 and state[i,j-1] == 1:
                continue
                
            state[i,j] = 2
            B -= 1
            if B <= 0:
                break
                
    if B > 0:
        return []
    
    return state

s = minimize_surface(5, 5, 11)
fill_state(s, 5)

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 0., 0., 0., 0.],
       [0., 2., 2., 2., 2.],
       [2., 0., 0., 0., 0.]])

In [30]:
s = minimize_surface(5, 5, 11)
fill_state(s, 9)

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 0., 0., 0., 0.],
       [0., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.]])

In [31]:
s = minimize_surface(5, 5, 11)
fill_state(s, 10)

[]

It is filling correctly. Let me implement the function seat and change the type of the array to integers.

In [37]:
from typing import List

def minimize_surface(R, C, A, state=None):
    
    if state is None:
        state = zeros([R, C], dtype=int)
        
    state[0,0] = 1
    
    for i in range(1, A):
        surface_idx = surface(state)
        surface_list = []
        
        for s in surface_idx:
            
            state[s[0],s[1]] = 1
            surface_list.append(calculate_surface(state))
            state[s[0],s[1]] = 0
            
        s_min = surface_list.index(min(surface_list))
        s_min = surface_idx[s_min]
        
        state[s_min[0], s_min[1]] = 1
        
    return state

def seat(R: int, C: int, A: int, B: int) -> List[str]:
    
    if A+B >= R*C:
        return []
    
    D = min(A, B)
    
    state = minimize_surface(R, C, D)
    s = calculate_surface(state)
    
    E = max(A, B)
    s = fill_state(state, E)
    
    if isinstance(s, list):
        return []
    
    res = []
    D_char = 'A' if D == A else 'B'
    E_char = 'A' if E == A else 'B'
    for row in state:
        row = [str(r) for r in row]
        r = ''.join(row)
        r.replace('1', D_char)
        r.replace('2', E_char)
        r.replace('-', '.')
        res.append(r)
        
    return res

seat(4,6,2,3)

['110222', '002000', '200000', '200000']

The replace is not working because I forgot to add r = r.replace. I also need to change '-' by '0' in the last replace. 

In [42]:
def seat(R: int, C: int, A: int, B: int) -> List[str]:
    
    if A+B >= R*C:
        return []
    
    D = min(A, B)
    
    state = minimize_surface(R, C, D)
    s = calculate_surface(state)
    
    E = max(A, B)
    s = fill_state(state, E)
    
    if isinstance(s, list):
        return []
    
    res = []
    D_char = 'A' if D == A else 'B'
    E_char = 'A' if E == A else 'B'
    for row in state:
        row = [str(r) for r in row]
        r = ''.join(row)
        r = r.replace('1', D_char)
        r = r.replace('2', E_char)
        r = r.replace('0', '.')
        res.append(r)
        
    return res

state = seat(4,6,2,3)
for s in state:
    print(s)

AA.BBB
..B...
B.....
B.....


The result is incorrect. Let me add a few prints to check the variable state

In [45]:
def seat(R: int, C: int, A: int, B: int) -> List[str]:
    
    if A+B >= R*C:
        return []
    
    D = min(A, B)
    
    state = minimize_surface(R, C, D)
    print(state)
    
    E = max(A, B)
    state = fill_state(state, E)
    print(state)
    
    if isinstance(state, list):
        return []
    
    res = []
    D_char = 'A' if D == A else 'B'
    E_char = 'A' if E == A else 'B'
    for row in state:
        row = [str(r) for r in row]
        r = ''.join(row)
        r = r.replace('1', D_char)
        r = r.replace('2', E_char)
        r = r.replace('0', '.')
        res.append(r)
        
    return res

state = seat(4,6,2,3)
for s in state:
    print(s)

[[1 1 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[[1 1 0 2 2 2]
 [0 0 2 0 0 0]
 [2 0 0 0 0 0]
 [2 0 0 0 0 0]]
AA.BBB
..B...
B.....
B.....


The problem seems to be in fill_state

In [47]:
def fill_state(state, B):
    
    x, y = state.shape
    
    for i in range(x):
        for j in range(y):
            if state[i,j] == 1:
                continue
                
            if i+1 < x  and state[i+1,j  ] == 1:
                continue
            if j+1 < y  and state[i  ,j+1] == 1:
                continue
            if i-1 >= 0 and state[i-1,j] == 1:
                continue
            if j-1 >= 0 and state[i,j-1] == 1:
                continue
                
            print(i, j, B)
            state[i,j] = 2
            B -= 1
            if B <= 0:
                break
                
    if B > 0:
        return []
    
    return state

state = zeros([4, 6])
state[0,0] = state[1,0] = 1
print(state)
fill_state(state, 3)

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
0 2 3
0 3 2
0 4 1
1 2 0
2 1 -1
3 0 -2


array([[1., 0., 2., 2., 2., 0.],
       [1., 0., 2., 0., 0., 0.],
       [0., 2., 0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0.]])

The break must be in the x loop as well. 

In [48]:
def fill_state(state, B):
    
    x, y = state.shape
    
    for i in range(x):
        
        if B <= 0:
                break
                
        for j in range(y):
            if state[i,j] == 1:
                continue
                
            if i+1 < x  and state[i+1,j  ] == 1:
                continue
            if j+1 < y  and state[i  ,j+1] == 1:
                continue
            if i-1 >= 0 and state[i-1,j] == 1:
                continue
            if j-1 >= 0 and state[i,j-1] == 1:
                continue
                
            state[i,j] = 2
            B -= 1
            
            if B <= 0:
                break
                
    if B > 0:
        return []
    
    return state

def seat(R: int, C: int, A: int, B: int) -> List[str]:
    
    if A+B >= R*C:
        return []
    
    D = min(A, B)
    
    state = minimize_surface(R, C, D)
    
    E = max(A, B)
    state = fill_state(state, E)
    
    if isinstance(state, list):
        return []
    
    res = []
    D_char = 'A' if D == A else 'B'
    E_char = 'A' if E == A else 'B'
    for row in state:
        row = [str(r) for r in row]
        r = ''.join(row)
        r = r.replace('1', D_char)
        r = r.replace('2', E_char)
        r = r.replace('0', '.')
        res.append(r)
        
    return res

state = seat(4,6,2,3)
for s in state:
    print(s)

AA.BBB
......
......
......


Now it is correct. Let me write a function to test and print the state

In [54]:
def test(*args):
    state = seat(*args)
    
    if len(state) == 0:
        print(state)
        return
    
    for s in state:
        print(s)

test(4,5,13,7)

[]


In [55]:
test(4,5,14,2)

BB.AA
..AAA
AAAAA
AAAA.


correct 

In [56]:
test(5,5,17,4)

BBB.A
B..AA
.AAAA
AAAAA
AAAAA


Correct.

Let me test time

In [58]:
from datetime import datetime

t1 = datetime.now()
test(50,50,50*50//2,50*50//2)
t2 = datetime.now()
print(t2 - t1)

[]
0:00:00.000113


doing a for loop the test the wort case

In [61]:
for size in range(1, 51):
    t1 = datetime.now()
    test(size,size,size**2//2,size**2//2//2-size)
    t2 = datetime.now()
    print(size, t2 - t1)
    print('------------------------------')

B
1 0:00:00.000090
------------------------------
[]
2 0:00:00.000020
------------------------------
B.A
.AA
A..
3 0:00:00.000044
------------------------------
B.AA
.AAA
AAA.
....
4 0:00:00.000057
------------------------------
B.AAA
.AAAA
AAAAA
.....
.....
5 0:00:00.000073
------------------------------
BB.AAA
B.AAAA
.AAAAA
AAAAAA
......
......
6 0:00:00.000412
------------------------------
BBB.AAA
BB.AAAA
..AAAAA
AAAAAAA
AAAAA..
.......
.......
7 0:00:00.001066
------------------------------
BBBB.AAA
BBB.AAAA
B..AAAAA
.AAAAAAA
AAAAAAAA
AAAAA...
........
........
8 0:00:00.002445
------------------------------
BBBBB.AAA
BBB..AAAA
BB.AAAAAA
B.AAAAAAA
.AAAAAAAA
AAAAAAAAA
AAA......
.........
.........
9 0:00:00.005009
------------------------------
BBBBB.AAAA
BBBB.AAAAA
BBB.AAAAAA
BB.AAAAAAA
B.AAAAAAAA
.AAAAAAAAA
AAAAAAAAAA
A.........
..........
..........
10 0:00:00.009243
------------------------------
BBBBBB.AAAA
BBBBB.AAAAA
BBBB.AAAAAA
BBB.AAAAAAA
B..AAAAAAAA
.AAAAAAAAAA
AAAAAAAAAA

KeyboardInterrupt: 

I stopped the loop because time seems to be increasing just fine.
Let me start from 40.

In [62]:
for size in range(40, 51):
    t1 = datetime.now()
    test(size,size,size**2//2,size**2//2//2-size)
    t2 = datetime.now()
    print(size, t2 - t1)
    print('------------------------------')

BBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBB..AAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
BBB.AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
BB.AAAAAAAAAAAAA

KeyboardInterrupt: 

In [63]:
size = 50

t1 = datetime.now()
test(size,size,size**2//2,size**2//2//2-size)
t2 = datetime.now()

print(size, t2 - t1)

BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBB..AAAAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBB.AAAAAAAAAAAAAAAA

There are two //2//2 instead of a single //2. Let me fix it and test the time againg. 

In [66]:
for size in range(40, 51):
    t1 = datetime.now()
    test(size,size,size**2//2,size**2//2-size)
    t2 = datetime.now()
    print(size, t2 - t1)
    print('------------------------------')

BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB......A
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBB.A

KeyboardInterrupt: 

In [67]:
print('should take less than 2 min')

size = 50

t1 = datetime.now()
test(size,size,size**2//2,size**2//2-size)
t2 = datetime.now()
print(size, t2 - t1)

should take less than 2 min
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB...
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.....AAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAA
BBB

I miss 1 edge case. The case when C > R. The algorithm should fill the side with the least ammount first.

In [70]:
def seat(R: int, C: int, A: int, B: int) -> List[str]:
    
    if A+B >= R*C:
        return []
    
    if C > R:
        inv = seat(C, R, A, B)
        res = ['' for i in range(R)]
        for i in range(C):
            for j in range(R):
                res[j] += inv[i][j]
        return res
    
    D = min(A, B)
    
    state = minimize_surface(R, C, D)
    
    E = max(A, B)
    state = fill_state(state, E)
    
    if isinstance(state, list):
        return []
    
    res = []
    D_char = 'A' if D == A else 'B'
    E_char = 'A' if E == A else 'B'
    for row in state:
        row = [str(r) for r in row]
        r = ''.join(row)
        r = r.replace('1', D_char)
        r = r.replace('2', E_char)
        r = r.replace('0', '.')
        res.append(r)
        
    return res

state = seat(4,6,12,8)
for s in state:
    print(s)

BB.AAA
BB.AAA
BB.AAA
BB.AAA


In [71]:
state = seat(6,4,12,8)
for s in state:
    print(s)

BBBB
BBBB
....
AAAA
AAAA
AAAA


The results above seems correct and now we fill the smallest side first. 

In [74]:
state = seat(4,3,4,4)
for s in state:
    print(s)

AAA
A..
.AA
AA.


I forgot to cover the case when A == B.
I will just add a if to change the value of E_char

In [75]:
def seat(R: int, C: int, A: int, B: int) -> List[str]:
    
    if A+B >= R*C:
        return []
    
    if C > R:
        inv = seat(C, R, A, B)
        res = ['' for i in range(R)]
        for i in range(C):
            for j in range(R):
                res[j] += inv[i][j]
        return res
    
    D = min(A, B)
    
    state = minimize_surface(R, C, D)
    
    E = max(A, B)
    state = fill_state(state, E)
    
    if isinstance(state, list):
        return []
    
    res = []
    D_char = 'A' if D == A else 'B'
    E_char = 'A' if E == A else 'B'
    if A == B:
        E_char = 'B'
        
    for row in state:
        row = [str(r) for r in row]
        r = ''.join(row)
        r = r.replace('1', D_char)
        r = r.replace('2', E_char)
        r = r.replace('0', '.')
        res.append(r)
        
    return res

state = seat(4,3,4,4)
for s in state:
    print(s)

AAA
A..
.BB
BB.


Now it looks correct. 

Let me do the final test

In [76]:
print('should take less than 2 min')

size = 50

t1 = datetime.now()
test(size,size,size**2//2,size**2//2-size)
t2 = datetime.now()
print(size, t2 - t1)

should take less than 2 min
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB...
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.....AAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBBBBBBBBBBBBBBBBBB.AAAAAAAAAAAAAAAAAAA
BBB